# PyNEMA2
[PyNMEA2](https://github.com/Knio/pynmea2) is a python library for the NMEA 0183 protocol.

## Parsing NMEA messages

In [1]:
import pynmea2

**GGA** -  GPS Fix Data

In [2]:
msg = pynmea2.parse("$GPGGA,184353.07,1929.045,S,02410.506,E,1,04,2.6,100.00,M,-33.9,M,,0000*6D", check=True)
msg

<GGA(timestamp=datetime.time(18, 43, 53, 70000), lat='1929.045', lat_dir='S', lon='02410.506', lon_dir='E', gps_qual=1, num_sats='04', horizontal_dil='2.6', altitude=100.0, altitude_units='M', geo_sep='-33.9', geo_sep_units='M', age_gps_data='', ref_station_id='0000')>

The `lat` and `lon` attributes are in DDDMM.SSSSS format while `latitude` and `longitude` are their float values.

In [10]:
msg.lat, msg.latitude, msg.latitude_minutes, msg.latitude_seconds

('1929.045', -19.484083333333334, 29.045000000000073, 2.6999999999970896)

In [11]:
msg.lon, msg.longitude, msg.longitude_minutes, msg.longitude_seconds

('02410.506', 24.1751, 10.506000000000085, 30.360000000000582)

**VTG** - Track Made Good and Ground Speed

In [4]:
pynmea2.parse("$GPVTG,054.7,T,034.4,M,005.5,N,010.2,K*48")

<VTG(true_track=54.7, true_track_sym='T', mag_track=Decimal('34.4'), mag_track_sym='M', spd_over_grnd_kts=Decimal('5.5'), spd_over_grnd_kts_sym='N', spd_over_grnd_kmph=10.2, spd_over_grnd_kmph_sym='K')>

**ZDA** - Time and Date

In [13]:
pynmea2.parse("$GPZDA,201530.00,04,07,2002,00,00*60")    # Time is in UTC, or GPS time if offset is not yet known

<ZDA(timestamp=datetime.time(20, 15, 30), day=4, month=7, year=2002, local_zone=0, local_zone_minutes=0)>

## Generating NMEA messages

In [6]:
msg = pynmea2.GGA('GP', 'GGA', ('184353.07', '1929.045', 'S', '02410.506', 'E', '1', '04', '2.6', 
                                '100.00', 'M', '-33.9', 'M', '', '0000'))
msg

<GGA(timestamp=datetime.time(18, 43, 53, 70000), lat='1929.045', lat_dir='S', lon='02410.506', lon_dir='E', gps_qual=1, num_sats='04', horizontal_dil='2.6', altitude=100.0, altitude_units='M', geo_sep='-33.9', geo_sep_units='M', age_gps_data='', ref_station_id='0000')>

In [7]:
str(msg)

'$GPGGA,184353.07,1929.045,S,02410.506,E,1,04,2.6,100.00,M,-33.9,M,,0000*6D'

**Notes**
* Check [talker.py](https://github.com/Knio/pynmea2/blob/master/pynmea2/types/talker.py) for reference.
* The `pynmea2.GGA` class and others require `lat` and `lon` values in DDDMM.SSSSS format as well as `lat_dir` and `lon_dir`. Use the following function to convert floats into that format (the inverse to nmea_utils.py:dm_to_sd()).

In [8]:
def sd_to_dm(latitude, longitude):
    if latitude < 0:
        lat_dir = 'S'
    else:
        lat_dir = 'N'
    lat = ('%010.5f' % (abs(int(latitude)) * 100 + (abs(latitude) % 1.0) * 60)).rstrip('0')
        
    if longitude < 0:
        lon_dir = 'W'
    else:
        lon_dir = 'E'
    lon = ('%011.5f' % (abs(int(longitude)) * 100 + (abs(longitude) % 1.0) * 60)).rstrip('0')
    
    return lat, lat_dir, lon, lon_dir

# 1929.045,S,02410.506,E
sd_to_dm(-19.484083333333334, 24.1751)

('1929.045', 'S', '02410.506', 'E')